In [1]:
import ee
import geemap
from geemap import ml
from sklearn import ensemble
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


# Inicializa la autenticación y la inicialización de Google Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-facuboladerasgee')

In [2]:
# Definir la región de interés
roi = ee.FeatureCollection('projects/ee-facuboladerasgee/assets/Betroka')
# Centrar el mapa en la región de interés
Map = geemap.Map(center=[0, 0], zoom=2)
Map.add_basemap('SATELLITE')
Map.centerObject(roi)

In [3]:
roi_feature = ee.FeatureCollection(roi).first()
roi_geometry = roi_feature.geometry()
roi_coordinates = roi_geometry.coordinates().getInfo()

# Guardar las coordenadas en una variable
roi_coords = roi_coordinates
ee_roi =  ee.Geometry.Polygon(roi_coords) 

In [11]:
year = 2015
start = f'{year}-01-01'
end = f'{year}-11-30'

l8_col = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(roi) \
    .filterDate(start, end) \
    .filter(ee.Filter.lt('CLOUD_COVER', 60))

def mask_clouds_landsat(image):
    qa = image.select('QA_PIXEL')
    cloud_shadow_bit_mask = ee.Number(2).pow(3).int()
    clouds_bit_mask = ee.Number(2).pow(5).int()
    mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0) \
        .And(qa.bitwiseAnd(clouds_bit_mask).eq(0))
    return image.updateMask(mask)


l8_masked = l8_col.map(mask_clouds_landsat)
image = l8_masked.median().toFloat().clip(roi)

ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')  # NIR y Red
mndwi = image.normalizedDifference(['SR_B3', 'SR_B6']).rename('MNDWI')  # Green y SWIR1
ndbi = image.normalizedDifference(['SR_B6', 'SR_B5']).rename('NDBI')  # SWIR1 y NIR
nbr = image.normalizedDifference(['SR_B5', 'SR_B7']).rename('NBR')  # NIR y SWIR2
ndwi = image.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI')

rvi_opt = image.expression(
    '(NIR / RED)',
    {
        'NIR': image.select('SR_B5'),
        'RED': image.select('SR_B4').where(image.select('SR_B4').eq(0), 0.0001)
    }
).rename('RVI')

evi = image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
    {
        'NIR': image.select('SR_B5'),  # NIR
        'RED': image.select('SR_B4'),  # Red
        'BLUE': image.select('SR_B2')  # Blue
    }
).rename('EVI')

savi = image.expression(
    '((NIR - RED) / (NIR + RED + 0.5)) * 1.5',
    {
        'NIR': image.select('SR_B5'),
        'RED': image.select('SR_B4')
    }
).rename('SAVI')

cvi = image.expression(
    'NIR * (RED / GREEN)',
    {
        'NIR': image.select('SR_B5'),
        'RED': image.select('SR_B4'),
        'GREEN': image.select('SR_B3')
    }
).rename('CVI')

image = image.addBands([ndvi, mndwi, ndbi, evi, savi, rvi_opt, nbr, ndwi, cvi])

# Dynamic World collection for label extraction
dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
    .filterBounds(roi) \
    .filterDate(start, end)

# Get the median label from Dynamic World
label = dw_col.select('label').median().rename('label')

# Add DEM, slope, and aspect from SRTM DEM
dem = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
slope = ee.Terrain.slope(dem).rename('slope').clip(roi)
aspect = ee.Terrain.aspect(dem).rename('aspect').clip(roi)

# Add elevation, slope, aspect, and label to the image
image = image.addBands([dem.rename('elevation'), slope, aspect, label])

In [12]:
import math

def terrain_correction_palsar(image, roi, angle_image):

    srtm = ee.Image('USGS/SRTMGL1_003').clip(roi)    
    # Convertir la imagen a dB usando la fórmula γ₀ = 10 log₁₀(DN²) - 83.0 dB
    dn_squared = image.pow(2)   
    backscatter = ee.Image.constant(10).multiply(dn_squared.log10()).subtract(83.0)
    theta_i = angle_image  
    
    # Obtener el aspecto promedio de la región de interés
    phi_i = ee.Terrain.aspect(theta_i).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,  # Usar la ROI
        scale=1000
    ).get('aspect')
    
    # Convertir phi_i a una imagen constante
    phi_i_image = ee.Image.constant(phi_i)
    
    # Geometría del terreno
    alpha_s = ee.Terrain.slope(srtm).select('slope')
    phi_s = ee.Terrain.aspect(srtm).select('aspect')
    
    # Geometría del modelo
    phi_r = phi_i_image.subtract(phi_s)
    
    # Convertir todo a radianes
    phi_r_rad = phi_r.multiply(math.pi / 180)
    alpha_s_rad = alpha_s.multiply(math.pi / 180)
    theta_i_rad = theta_i.multiply(math.pi / 180)
    ninety_rad = ee.Image.constant(90).multiply(math.pi / 180)

    # Pendiente en el rango
    alpha_r = (alpha_s_rad.tan().multiply(phi_r_rad.cos())).atan()
    # Pendiente en acimut
    alpha_az = (alpha_s_rad.tan().multiply(phi_r_rad.sin())).atan()

    # Ángulo de incidencia local
    theta_lia = (alpha_az.cos().multiply((theta_i_rad.subtract(alpha_r)).cos())).acos()
    theta_lia_deg = theta_lia.multiply(180 / math.pi)

    # Gamma_nought_flat usando la fórmula ajustada
    gamma0 = backscatter.divide(theta_i_rad.cos())
    gamma0_dB = gamma0.rename('gamma0_dB')

    # Modelo volumétrico
    nominator = (ninety_rad.subtract(theta_i_rad).add(alpha_r)).tan()
    denominator = (ninety_rad.subtract(theta_i_rad)).tan()
    vol_model = (nominator.divide(denominator)).abs()

    # Aplicar el modelo
    gamma0_volume = gamma0.divide(vol_model)
    gamma0_volume_dB = gamma0_volume.rename('gamma0_volume_dB')

    return gamma0_dB


def preprocess(image, roi, angle_image):
    image = terrain_correction_palsar(image, roi, angle_image)
    speckle_filtered = image.focal_median(kernelType='circle', radius=50, units='meters')
    return speckle_filtered

angle = (
    ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH')
    .filterBounds(roi)
    .filterDate(start, end)
    .select('angle')  
    .median() 
)

img_hh = (
    ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH')
    .filterBounds(roi)
    .filterDate(start, end)
    .select('HH')  # Seleccionar solo HH
    .map(lambda img: preprocess(img, roi, angle))  # Pasar roi y angle_image al preprocesamiento
)

img_hv = (
    ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH')
    .filterBounds(roi)
    .filterDate(start, end)
    .select('HV')  # Seleccionar solo HV
    .map(lambda img: preprocess(img, roi, angle))  # Pasar roi y angle_image al preprocesamiento
)

palsar_hh_median = img_hh.median().rename('HH_dB')
palsar_hv_median = img_hv.median().rename('HV_dB')

hh_plus_hv = palsar_hh_median.add(palsar_hv_median).rename('HH_plus_HV')
hh_minus_hv = palsar_hh_median.subtract(palsar_hv_median).rename('HH_minus_HV')
hh_div_hv = palsar_hh_median.divide(palsar_hv_median).rename('HH_div_HV')
rvi = palsar_hv_median.divide(palsar_hh_median.add(palsar_hv_median)).multiply(4).rename('RVI_palsar')

palsar_combined = ee.Image.cat([palsar_hh_median, palsar_hv_median, hh_plus_hv, hh_minus_hv, hh_div_hv, rvi]).clip(roi)
image = image.addBands(palsar_combined)

In [13]:
def calculate_texture_metrics(image, band_name, roi, size=3):
    
    band_int = image.select(band_name).toInt32().clip(roi)    
    # Calcular texturas usando la banda convertida a int32
    glcm = band_int.glcmTexture(size=size)
    
    # Extraer las texturas de interés y renombrarlas
    contrast = glcm.select(f'{band_name}_contrast').rename(f'{band_name}_Contrast')
    correlation = glcm.select(f'{band_name}_corr').rename(f'{band_name}_Correlation')
    entropy = glcm.select(f'{band_name}_ent').rename(f'{band_name}_Entropy')
    inertia = glcm.select(f'{band_name}_inertia').rename(f'{band_name}_Inertia')
    
    # Agregar las bandas de textura a la imagen original
    return image.addBands([contrast, correlation, entropy, inertia])


image = calculate_texture_metrics(image, 'HH_dB', roi)
image = calculate_texture_metrics(image, 'HV_dB', roi)


bands = [
    "EVI",
    "SR_B4",
    "slope",
    "elevation",
    "HH_minus_HV",
    "SR_B3",
    "aspect",
    "SR_B6",
    "HH_dB_Correlation",
    "HV_dB_Correlation",
    "HH_dB_Entropy",
    "SR_B2",
    "MNDWI",
    "NDBI",
    "HV_dB",   
]

image = image.select(bands)

In [14]:
Map.centerObject(roi, 8)
Map.addLayer(image.select('HV_dB'), {}, 'HV_dB', False)
Map.addLayer(image.select('SR_B4'), {}, 'SR_B4', False)


Map

Map(center=[-23.205899401827804, 46.08425652331407], controls=(WidgetControl(options=['position', 'transparent…

In [27]:
feature_names =  [
    "EVI",
    "SR_B4",
    "slope",
    "elevation",
    "HH_minus_HV",
    "SR_B3",
    "aspect",
    "SR_B6",
    "HH_dB_Correlation",
    "HV_dB_Correlation",
    "HH_dB_Entropy",
    "SR_B2",
    "MNDWI",
    "NDBI",
    "HV_dB",
   
]

label = "agbd"

user_id = 'users/facuboladerasgee'
# specify asset id where to save trees
# be sure to change  to your ee user name
asset_id = user_id + "/Betroka_filter"
asset_id

# read the exported tree feature collection
rf_fc = ee.FeatureCollection(asset_id)

# convert it to a classifier, very similar to the `ml.trees_to_classifier` function
another_classifier = ml.fc_to_classifier(rf_fc)

# classify the image again but with the classifier from the persisted trees
classified = image.select(feature_names).classify(another_classifier)



In [35]:
# # Feature names for classification
# feature_names = [
#     "SR_B4",
#     "slope",
#     "elevation",
#     "NDBI",
#     "EVI",
#     "SR_B3",
#     "NDWI",
#     "aspect",
#     "SR_B2",
#     "HV_dB_Entropy",
#     "HV_dB_Correlation",
#     "NBR",
#     "HH_dB_Correlation",
#     "HH_minus_HV",
#     "HV_dB",
#     "HH_dB_Entropy",
#     "HH_dB",
#     "MNDWI",
#     "RVI_palsar",
#     "HH_plus_HV"
# ]

# label = "agbd"

# # Define user ID and asset ID for the Random Forest feature collection
# user_id = 'users/facuboladerasgee'
# asset_id = f'{user_id}/mdg_south'

# # Read the exported tree feature collection (Random Forest model)
# rf_fc = ee.FeatureCollection(asset_id)

# # Convert the FeatureCollection into a classifier (custom function needed)
# # Assuming 'ml.fc_to_classifier' is a custom function in your script:
# another_classifier = ml.fc_to_classifier(rf_fc)

# # Classify the image with the classifier from the persisted trees
# classified = image.select(feature_names).classify(another_classifier)

# # Get the Dynamic World layer to filter out water (label = 0)
# dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
#     .filterBounds(roi) \
#     .filterDate(start, end) \
#     .select('label')  # Select the label band

# # Get the median Dynamic World label
# dw_label = dw_col.median()

# # Create a mask where label is not equal to 0 (water)
# no_water_mask = dw_label.neq(0)

# # Apply the mask to the image, which removes water areas
# image = image.updateMask(no_water_mask)

# # Now the image is masked, and you can proceed with texture metrics
# image = calculate_texture_metrics(image, 'HH_dB', roi)
# image = calculate_texture_metrics(image, 'HV_dB', roi)

# # Select the bands for classification
# bands = [
#     "SR_B4",
#     "slope",
#     "elevation",
#     "NDBI",
#     "EVI",
#     "SR_B3",
#     "NDWI",
#     "aspect",
#     "SR_B2",
#     "HV_dB_Entropy",
#     "HV_dB_Correlation",
#     "NBR",
#     "HH_dB_Correlation",
#     "HH_minus_HV",
#     "HV_dB",
#     "HH_dB_Entropy",
#     "HH_dB",
#     "MNDWI",
#     "RVI_palsar",
#     "HH_plus_HV"
# ]

# image = image.select(bands)

# # Perform classification with the classifier
# classified1 = image.select(feature_names).classify(another_classifier)

In [15]:
# Definir los nombres de las características (features)
feature_names = [
    "EVI",
    "SR_B4",
    "slope",
    "elevation",
    "HH_minus_HV",
    "SR_B3",
    "aspect",
    "SR_B6",
    "HH_dB_Correlation",
    "HV_dB_Correlation",
    "HH_dB_Entropy",
    "SR_B2",
    "MNDWI",
    "NDBI",
    "HV_dB",   
]


label = "agbd"

# ID del usuario en GEE
user_id = 'users/facuboladerasgee'

# Cargar la colección Dynamic World para obtener la máscara de agua
dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
    .filterBounds(roi) \
    .filterDate('2023-01-01', '2023-06-01') \
    .select('label')  # Seleccionar la banda 'label' que contiene las clases

# Obtener la mediana de las etiquetas de Dynamic World
dw_label = dw_col.median()

# Crear una máscara donde la etiqueta no sea igual a 0 (0 = agua)
no_water_mask = dw_label.neq(0)

# Aplicar la máscara de agua a la imagen, eliminando áreas con agua
masked_image = image.updateMask(no_water_mask)

# Iterar sobre los 10 modelos que has subido a GEE
for i in range(1, 11):
    # Especificar el asset_id del modelo i
    asset_id = f"{user_id}/Rf_Bootstrap_Betroka__model_{i}"
    
    # Leer la colección de características del árbol exportado
    rf_fc = ee.FeatureCollection(asset_id)
    
    # Convertirlo a un clasificador (ml.fc_to_classifier es una función personalizada)
    classifier = ml.fc_to_classifier(rf_fc)
    
    # Clasificar la imagen con el clasificador actual, usando la imagen sin agua
    classified = masked_image.select(feature_names).classify(classifier)
    
    # Almacenar cada predicción en una variable individual (opcional)
    globals()[f'classified_{i}'] = classified
    
    # Confirmación de que el modelo fue usado
    print(f"Modelo {i} clasificado y almacenado en la variable classified_{i}")


Modelo 1 clasificado y almacenado en la variable classified_1
Modelo 2 clasificado y almacenado en la variable classified_2
Modelo 3 clasificado y almacenado en la variable classified_3
Modelo 4 clasificado y almacenado en la variable classified_4
Modelo 5 clasificado y almacenado en la variable classified_5
Modelo 6 clasificado y almacenado en la variable classified_6
Modelo 7 clasificado y almacenado en la variable classified_7
Modelo 8 clasificado y almacenado en la variable classified_8
Modelo 9 clasificado y almacenado en la variable classified_9
Modelo 10 clasificado y almacenado en la variable classified_10


In [17]:
# Centrar el mapa en la región de interés
Map = geemap.Map(center=[0, 0], zoom=2)
Map.centerObject(roi)

visPredictedBiomass = {
    'min': 0,
    'max': 200,
    'palette': ['lightyellow', 'lightgreen', 'green', 'darkgreen']
}

Map = geemap.Map(zoom=11)
Map.add_basemap('SATELLITE')
Map.centerObject(roi)

Map.addLayer(
    classified_1,
    visPredictedBiomass,
    "classification",)


Map.add_colorbar(visPredictedBiomass, label="Predicted Biomass")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [54]:
import time

# Función para exportar cada clasificador a Google Drive
def export_classified_to_drive(classified_image, description, folder, region, scale=100, crs='EPSG:4326'):
    task = ee.batch.Export.image.toDrive(
        image=classified_image,
        description=description,
        folder=folder,
        region=ee_roi,
        scale=100,
        crs=crs
    )
    task.start()
    
    print(f"Exportación de {description} iniciada.")
    
    # Bucle para esperar hasta que la tarea se complete correctamente o falle definitivamente
    while task.active() or task.status()['state'] == 'READY' or task.status()['state'] == 'RUNNING':
        print(f'Exportación de {description} en progreso...')
        time.sleep(30)  # Esperar 30 segundos antes de verificar de nuevo el estado

    if task.status()['state'] == 'COMPLETED':
        print(f'Exportación de {description} completada con éxito.')
    else:
        print(f'Error en la exportación de {description}:', task.status())
        return False  # Devolver False si la tarea falló

    return True  # Devolver True si la tarea fue exitosa

# Iterar sobre los 10 clasificadores y exportarlos
for i in range(1,11):
    classified_image = globals()[f'classified_{i}']  # Obtener la variable classified_i
    description = f'Rf_Bootstrap_Betroka__model_{i}'  # Nombre de la tarea de exportación
    folder = 'Rf_Bootstrap_Betroka_2015'  # Carpeta de Google Drive
    
    success = export_classified_to_drive(classified_image, description, folder, ee_roi)
    
    if not success:
        print(f'La exportación del modelo {i} falló. Intentar nuevamente.')
        break  # Romper el bucle si hay un fallo en la exportación

print("Proceso de exportación completado.")

Exportación de Rf_Bootstrap_Betroka__model_1 iniciada.
Exportación de Rf_Bootstrap_Betroka__model_1 en progreso...
Exportación de Rf_Bootstrap_Betroka__model_1 en progreso...
Exportación de Rf_Bootstrap_Betroka__model_1 en progreso...
Exportación de Rf_Bootstrap_Betroka__model_1 en progreso...
Exportación de Rf_Bootstrap_Betroka__model_1 en progreso...
Exportación de Rf_Bootstrap_Betroka__model_1 en progreso...
Exportación de Rf_Bootstrap_Betroka__model_1 en progreso...
Exportación de Rf_Bootstrap_Betroka__model_1 completada con éxito.
Exportación de Rf_Bootstrap_Betroka__model_2 iniciada.
Exportación de Rf_Bootstrap_Betroka__model_2 en progreso...
Exportación de Rf_Bootstrap_Betroka__model_2 en progreso...
Exportación de Rf_Bootstrap_Betroka__model_2 en progreso...
Exportación de Rf_Bootstrap_Betroka__model_2 en progreso...
Exportación de Rf_Bootstrap_Betroka__model_2 en progreso...
Exportación de Rf_Bootstrap_Betroka__model_2 en progreso...
Exportación de Rf_Bootstrap_Betroka__model_

In [29]:
import time

# Parámetros de exportación
task = ee.batch.Export.image.toDrive(
    image=classified,
    description='Betroka_filter',
    folder='Betroka',
    region=ee_roi,
    scale=100,
    crs='EPSG:4326',  # CRS adecuado para Nepal
    
    
)
task.start()

print("Exportación iniciada. Revisa tu Google Drive en la carpeta 'EE_RF'.")

# Esperar a que la tarea de exportación se complete (opcional)
while task.active():
    print('Exportación en progreso...')
    time.sleep(30)  # Esperar 30 segundos antes de verificar el estado nuevamente

if task.status()['state'] == 'COMPLETED':
    print('Exportación completada con éxito.')
else:
    print('Error en la exportación:', task.status())

Exportación iniciada. Revisa tu Google Drive en la carpeta 'EE_RF'.
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación completada con éxito.
